In [30]:
import numpy as np
from common.functions import softmax, cross_entropy_error

In [9]:
# 乗算レイヤの実装
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    
    def backward(self, dout): # 微分
        dx = dout * self.y # xとyをひっくり返す
        dy = dout * self.x
        return dx, dy

In [18]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print('apple price:', apple_price)


# backforward
dprice = 1
dapple_price , dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(f"dapple:{dapple}\ndapple_num:{dapple_num}\ndtax{dtax}")

apple price: 200
dapple:2.2
dapple_num:110.00000000000001
dtax200


In [19]:
# 加算レイヤnの実装

class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [27]:
# りんご2個とみかん3個の買い物

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orage_layter = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orage_layter.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_praice, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orage_layter.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## Reluレイヤの実装

In [28]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self):
        self.mask = (x <= 0)　# => boolのndarray
        out = x.copy()
        out[self.mask] == 0 #x(ndarray)の0以下の値を0にする
        
        return out
    
    
    def backward(self, dout):
        dout[self.mask] = 0 #maskの部分は0以下で微分も0, それ以外の　微分は1なので
        dx = dout
        
        return dx

## Sigmoidレイヤの実装

In [29]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        
        return dx
    

In [ ]:
## Affineレイヤの実装
class Affine:
    def __init__(self):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

## softmax and cross-entropyレイヤの実装 

In [31]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # １
        return dx